In [289]:
import os
import glob
import xarray as xr
import netCDF4 as nc
import numpy as np
from pyproj import Proj, Transformer, CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import math
import pandas as pd
import dask.array as da

In [327]:
current_dir = os.getcwd()

# WFDE5 data
wfde5_path = current_dir + '/shyft_workspace_copy/shyft_workspace/shyft-data/netcdf/orchestration-testdata/WFDE5/'
psurf_path = glob.glob(wfde5_path + 'psurf/*.nc')
qair_path = glob.glob(wfde5_path + 'qair/*.nc')
rainf_path = glob.glob(wfde5_path + 'rainf/*.nc')
swdown_path = glob.glob(wfde5_path + 'swdown/*.nc')
tair_path = glob.glob(wfde5_path + 'tair/*.nc')
wind_path = glob.glob(wfde5_path + 'wind/*.nc')
cell_data_path = glob.glob(wfde5_path + 'cell_data/*.nc')
asurf_path = glob.glob(wfde5_path + 'asurf/*.nc')

# Testdata
test_path = current_dir + '/shyft_workspace_copy/shyft_workspace/shyft-data/netcdf/orchestration-testdata/'
discharge_test_path = test_path + 'discharge.nc'
relhum_test_path = test_path + 'relative_humidity.nc'
precip_test_path = test_path + 'precipitation.nc'
swdown_test_path = test_path + 'radiation.nc'
temp_test_path = test_path + 'temperature.nc'
wind_test_path = test_path + 'wind_speed.nc'
cell_data_test_path = test_path + 'cell_data.nc'

In [328]:
psurf = xr.open_mfdataset(psurf_path)
qair = xr.open_mfdataset(qair_path)
rainf = xr.open_mfdataset(rainf_path)
swdown = xr.open_mfdataset(swdown_path)
tair = xr.open_mfdataset(tair_path)
wind = xr.open_mfdataset(wind_path)
cell_data = xr.open_mfdataset(cell_data_path)
asurf = xr.open_mfdataset(asurf_path)

discharge_test = xr.open_mfdataset(discharge_test_path)
relhum_test = xr.open_mfdataset(relhum_test_path)
precip_test = xr.open_mfdataset(precip_test_path)
swdown_test = xr.open_mfdataset(swdown_test_path)
temp_test = xr.open_mfdataset(temp_test_path)
wind_test = xr.open_mfdataset(wind_test_path)
cell_data_test = xr.open_mfdataset(cell_data_test_path)

In [54]:
wind_test

<xarray.Dataset>
Dimensions:      (time: 8760, station: 10)
Coordinates:
  * time         (time) datetime64[ns] 2013-09-01 ... 2014-08-31T23:00:00
    x            (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    y            (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    z            (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>
Dimensions without coordinates: station
Data variables:
    series_name  (station) object dask.array<chunksize=(10,), meta=np.ndarray>
    crs          int32 ...
    wind_speed   (time, station) float64 dask.array<chunksize=(8760, 10), meta=np.ndarray>

In [6]:
wind.lon.values[23]

72.75

## Convert lat and lon to UTM

In [332]:
## Make dataset where lat, lon have the same dimensions

asurf = asurf.sel(lon = asurf.lon.values[11:58])

## Extract lat and lon

lon = asurf.lon.values
lat = asurf.lat.values


## Find UTM CRS
utm_crs_list = query_utm_crs_info(
    datum_name="WGS 84",
    area_of_interest=AreaOfInterest(
        west_lon_degree=np.min(lon),
        south_lat_degree=np.min(lat),
        east_lon_degree=np.max(lon),
        north_lat_degree=np.max(lat),
    ),
)

utm_crs = CRS.from_epsg(utm_crs_list[0].code)
utm_crs

<Derived Projected CRS: EPSG:32642>
Name: WGS 84 / UTM zone 42N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 66°E and 72°E, northern hemisphere between equator and 84°N, onshore and offshore. Afghanistan. India. Kazakhstan. Kyrgyzstan. Pakistan. Russian Federation. Tajikistan. Uzbekistan.
- bounds: (66.0, 0.0, 72.0, 84.0)
Coordinate Operation:
- name: UTM zone 42N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [333]:
## Making a pyproj object for UTM Zone 41N

crs_4326 = CRS.from_epsg(4326)
crs_32642 = CRS.from_epsg(32642)

## Create transformer to convert from CRS to CRS

transformer = Transformer.from_crs(crs_4326, crs_32642, always_xy = True)

x, y = transformer.transform(lon, lat)


In [334]:
# Create a meshgrid

lon = asurf.lon.values
lat = asurf.lat.values
lonlon, latlat = np.meshgrid(lon, lat)

## Create new netCDF files on Shyft format

In [130]:
wind_test.station.dims

('station',)

In [111]:
time = wind.time.values


In [116]:
z = asurf.ASurf.values

In [173]:
wind_test.station.x.data

dask.array<open_dataset-86a6c42da34770dbbda4510951f590f1x, shape=(10,), dtype=float64, chunksize=(10,), chunktype=numpy.ndarray>

In [174]:
wind_test.station.x.data.compute()

array([331991., 299423., 339385., 365994., 332801., 296048., 308584.,
       355051., 273111., 338670.])

In [200]:
wind_test.station.x.coords

Coordinates:
    x        (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    y        (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    z        (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>

In [201]:
xx, yy, zz = np.meshgrid(x, y, z)

In [179]:
fg = da.from_array(wind_test.station.x.data.compute(), chunks = 1)
fg

dask.array<array, shape=(10,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>

In [225]:
wind_test.dims

Frozen({'time': 8760, 'station': 10})

In [210]:
np.shape(xx)

(47, 47, 2209)

In [215]:
stationd = xr.DataArray(
    data = np.arange(0,len(lon)),
    dims = ['station'],
    coords = dict(
    station = (['station'], np.arange(0,len(lon)),
    )
))

In [216]:
stationd

<xarray.DataArray (station: 47)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46])
Coordinates:
  * station  (station) int64 0 1 2 3 4 5 6 7 8 9 ... 38 39 40 41 42 43 44 45 46

In [212]:
stationd

<xarray.DataArray 'array-d2b609da69e4f2bfb58cac00903efb2f' (station: 47)>
dask.array<array, shape=(47,), dtype=int64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * station  (station) int64 0 1 2 3 4 5 6 7 8 9 ... 38 39 40 41 42 43 44 45 46

In [161]:
xd

<xarray.DataArray (dim_0: 47)>
array([ 259518.08748486,  313454.78105211,  367114.8752873 ,
        420490.04487269,  473571.82152148,  526351.58563029,
        578820.55830739,  630969.79378939,  682790.17226037,
        734272.39308909,  785406.96850088,  836184.21770233,
        886594.26147843,  936627.01728251,  986272.19484125,
       1035519.2922976 , 1084357.59291603, 1132776.16237522,
       1180763.84667443, 1228309.27068049, 1275400.83734319,
       1322026.72760723, 1368174.90104934, 1413833.09726961,
       1458988.83806569, 1503629.43041871, 1547741.9703191 ,
       1591313.34746016, 1634330.25082569, 1676779.17519765,
       1718646.42860741, 1759918.14075309, 1800580.27240284,
       1840618.6258017 , 1880018.85609673, 1918766.48379194,
       1956846.90824114, 1994245.42218262, 2030947.22731569,
       2066937.45091422, 2102201.16346743, 2136723.3973331 ,
       2170489.16638241, 2203483.48661015, 2235691.39767765,
       2267097.98534964, 2297688.40477967])
Dimensions without coordinates: dim_0

In [145]:
station = xr.DataArray(
    data = np.arange(0,47),
    dims = ['station',],
    coords = dict(
        x = (['station'], x),
        y = (['station'], y),
        z = (['station'], z),
    )
)



In [146]:
station

<xarray.DataArray (station: 47)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46])
Dimensions without coordinates: station

In [104]:
ds = xr.Dataset(
    {"wind_speed": (("time", "station"), wind.Wind.values)},
    coords= dict(
        "station" : np.arange(0,47),
        "time" : wind.time.values,
        "x": (["station"], x),
        "y": (["station"], y),
        "z": (["station"], asurf.ASurf.values),
    ),
)


#ds.to_netcdf("saved_on_disk.nc")

ValueError: Could not convert tuple of form (dims, data[, attrs, encoding]): (('time', 'station'), array([[[      nan,       nan,       nan, ..., 4.286641 , 4.907403 ,
         4.5442214],
        [      nan,       nan,       nan, ..., 3.902116 , 3.7727368,
         3.351109 ],
        [      nan,       nan,       nan, ..., 2.595103 , 2.182375 ,
         2.3332283],
        ...,
        [2.7922816, 2.9093125, 3.2171726, ..., 4.2986283, 4.5885477,
         4.292818 ],
        [2.1776001, 2.2702012, 2.5637178, ..., 5.386903 , 4.9357433,
         4.455282 ],
        [1.8251885, 1.6561942, 1.6775818, ..., 5.8242908, 5.282282 ,
         4.7171645]],

       [[      nan,       nan,       nan, ..., 4.613258 , 5.2152786,
         4.893939 ],
        [      nan,       nan,       nan, ..., 4.1317234, 4.071624 ,
         3.7351797],
        [      nan,       nan,       nan, ..., 2.854458 , 2.3545184,
         2.461487 ],
        ...,
        [2.7195554, 2.9323092, 3.2885528, ..., 3.8945544, 4.277

In [89]:
# Create new netcdf file on Shyft format

fn = test_path + '/test_files/test.nc'
ds = nc.Dataset(fn, 'w', format = 'NETCDF4')

In [91]:
# Add dimensions

time = ds.createDimension('time', None)
station = ds.createDimension('station', None)
xd = ds.createDimension('x', len(x))
yd = ds.createDimension('y', len(y))
zd = ds.createDimension('z', len(asurf.ASurf.values))


In [92]:
times = ds.createVariable('time', 'f8', ('time',))
xs = ds.createVariable('x', 'f8', ('x',))
ys = ds.createVariable('y', 'f8', ('y',))
zs = ds.createVariable('z', 'f8', ('z',))
value = ds.createVariable('value', 'f8', ('time', 'x', 'y', 'z'))
value.units = 'Unknown'

ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): time(0), station(0), x(47), y(47), z(47)
    variables(dimensions): float64 time(time), float64 x(x), float64 y(y), float64 z(z), float64 value(time, x, y, z)
    groups: 

In [95]:
len(asurf.ASurf.values)

47

In [93]:
zs[:] = asurf.ASurf.values
times[:] = wind.time.values
xs[:] = x
ys[:] = y

ValueError: input operand has more dimensions than allowed by the axis remapping

In [49]:
ds.close()

In [50]:
b = xr.open_mfdataset('/Users/jacobqs/Documents/MST Himalaya/MST-Himalaya/shyft_workspace_copy/shyft_workspace/shyft-data/netcdf/orchestration-testdata/test_files/test.nc')
b

<xarray.Dataset>
Dimensions:  (time: 157800, x: 47, y: 47, z: 47)
Coordinates:
  * time     (time) float32 9.151e+17 9.152e+17 ... 1.483e+18 1.483e+18
  * x        (x) float32 2.595e+05 3.135e+05 3.671e+05 ... 2.267e+06 2.298e+06
  * y        (y) float32 1.798e+06 1.853e+06 1.908e+06 ... 4.489e+06 4.556e+06
  * z        (z) float32 9.969e+36 9.969e+36 9.969e+36 ... 9.969e+36 9.969e+36
Data variables:
    value    (time, x, y, z) float32 dask.array<chunksize=(157800, 47, 47, 47), meta=np.ndarray>

In [39]:
ds.cre

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): time(0)
    variables(dimensions): 
    groups: 

# NEW TEST

In [261]:
wind_test.station

<xarray.DataArray 'station' (station: 10)>
array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Coordinates:
    x        (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    y        (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    z        (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>
Dimensions without coordinates: station

In [257]:
z = asurf.ASurf.values
xx, yy = np.meshgrid(x,y)
stations = np.arange(0,len(lon))
time = wind.time.values

In [322]:
wind_test.station.coords['z'].coords

Coordinates:
    x        (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    y        (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    z        (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>

In [323]:
wind_test.station.z.values

array([550., 160., 736., 860., 505., 460., 420., 860., 127., 627.])

In [326]:
asurf.ASurf.values[0]

array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  54., 263., 672., 629., 576., 542., 519., 446., 371.,
       334., 384., 530., 304., 129.,  42.,  10.,   3.,   2.,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan], dtype=float32)

In [337]:
a = wind_test.sel(station = 0).x.values
b = wind_test.sel(station = 1).x.values
c = wind_test.sel(station = 1).z.values
print(a)
print(b)
print(c)

331991.0
299423.0
160.0


In [335]:
asurf.ASurf

<xarray.DataArray 'ASurf' (lat: 47, lon: 47)>
dask.array<getitem, shape=(47, 47), dtype=float32, chunksize=(47, 47), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 16.25 16.75 17.25 17.75 ... 37.75 38.25 38.75 39.25
  * lon      (lon) float64 66.75 67.25 67.75 68.25 ... 88.25 88.75 89.25 89.75
Attributes:
    standard_name:  surface_altitude
    units:          m
    cell_methods:   area: mean
    cell_measures:  area: areacella
    long_name:      Surface Altitude
    type:           real

In [321]:
xd = xr.DataArray(
    data = da.from_array(x, chunks = len(x)),
    dims = ['station'],
    coords = dict(
        x = (['station'], da.from_array(x, chunks = len(x)) ),
        y = (['station'], da.from_array(y, chunks = len(y)) ),
        
    )
)

yd = xr.DataArray(
    data = da.from_array(y, chunks = len(y)),
    dims = ['station'],
    coords = dict(
        x = (['station'], da.from_array(x, chunks = len(x)) ),
        y = (['station'], da.from_array(y, chunks = len(y)) ),  
    )
)

In [341]:
xy_points = []
for i in range(len(x)):
    xy_points.append([x[i], y[i]])

In [347]:
z[46]

array([ 946., 1496., 2399., 2808., 2903., 3169., 3437., 2953., 2908.,
       3128., 3503., 3989., 4616., 4486., 4480., 4111., 3831., 2151.,
       1434., 1242., 1192., 1166., 1146., 1135., 1131., 1123., 1137.,
       1134., 1108., 1100., 1093., 1084., 1078., 1084., 1086., 1062.,
       1047., 1048., 1032., 1016.,  977.,  910.,  833.,  810.,  915.,
       1091., 1487.], dtype=float32)

In [352]:
z.shape

(47, 47)

In [348]:
xx, yy = np.meshgrid(x, y)
xy = np.stack([xx, yy], 2)
xy

array([[[ 259518.08748486, 1797911.10582747],
        [ 313454.78105211, 1797911.10582747],
        [ 367114.8752873 , 1797911.10582747],
        ...,
        [2235691.39767765, 1797911.10582747],
        [2267097.98534964, 1797911.10582747],
        [2297688.40477967, 1797911.10582747]],

       [[ 259518.08748486, 1852720.33723125],
        [ 313454.78105211, 1852720.33723125],
        [ 367114.8752873 , 1852720.33723125],
        ...,
        [2235691.39767765, 1852720.33723125],
        [2267097.98534964, 1852720.33723125],
        [2297688.40477967, 1852720.33723125]],

       [[ 259518.08748486, 1907641.52464692],
        [ 313454.78105211, 1907641.52464692],
        [ 367114.8752873 , 1907641.52464692],
        ...,
        [2235691.39767765, 1907641.52464692],
        [2267097.98534964, 1907641.52464692],
        [2297688.40477967, 1907641.52464692]],

       ...,

       [[ 259518.08748486, 4422985.47490395],
        [ 313454.78105211, 4422985.47490395],
        [ 367114.87528

In [303]:
xd.values

array([ 259518.08748486,  313454.78105211,  367114.8752873 ,
        420490.04487269,  473571.82152148,  526351.58563029,
        578820.55830739,  630969.79378939,  682790.17226037,
        734272.39308909,  785406.96850088,  836184.21770233,
        886594.26147843,  936627.01728251,  986272.19484125,
       1035519.2922976 , 1084357.59291603, 1132776.16237522,
       1180763.84667443, 1228309.27068049, 1275400.83734319,
       1322026.72760723, 1368174.90104934, 1413833.09726961,
       1458988.83806569, 1503629.43041871, 1547741.9703191 ,
       1591313.34746016, 1634330.25082569, 1676779.17519765,
       1718646.42860741, 1759918.14075309, 1800580.27240284,
       1840618.6258017 , 1880018.85609673, 1918766.48379194,
       1956846.90824114, 1994245.42218262, 2030947.22731569,
       2066937.45091422, 2102201.16346743, 2136723.3973331 ,
       2170489.16638241, 2203483.48661015, 2235691.39767765,
       2267097.98534964, 2297688.40477967])

In [273]:
da = xr.DataArray(
   data = stations,
   dims = ['station'],
   coords = dict(
    x = (['station'], xx),
    y = (['station'], yy),
    z = (['station'], zz),
   )
)

ValueError: different number of dimensions on data and dims: 1 vs 3

In [251]:
wind_test.time.dims

('time',)

In [235]:
z = asurf.ASurf.values
xx, yy = np.meshgrid(x,y)

In [245]:
time_darray = xr.DataArray(
    data = time,
    dims = ['time'],
    coords = dict(time = time)
)

In [255]:
time_darray

<xarray.DataArray (time: 157800)>
array(['1999-01-01T00:00:00.000000000', '1999-01-01T01:00:00.000000000',
       '1999-01-01T02:00:00.000000000', ...,
       '2016-12-31T21:00:00.000000000', '2016-12-31T22:00:00.000000000',
       '2016-12-31T23:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1999-01-01 ... 2016-12-31T23:00:00

In [254]:

ds = xr.Dataset(
        data = time_darray
)
ds

TypeError: Dataset.__init__() got an unexpected keyword argument 'data'